# Import

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from scipy.optimize import least_squares

# Read data

In [ ]:
# Read the CSV file
df = pd.read_csv(
    "/home/m3rc7pi/workspace/arc/log/calib/20240404_calib_full.csv", 
    header=None, 
    names=["timestamp", "acc_x_raw", "acc_y_raw", "acc_z_raw", "gyro_x_raw", "gyro_y_raw", "gyro_z_raw"],

)

range = (1 << 12)
scale_factor = 64

In [ ]:
# times 4 cause the offset is 8g, but acquisition is 2g
acc_x_raw_orig_offset = df['acc_x_raw'][0]
acc_y_raw_orig_offset = df['acc_y_raw'][0]
acc_z_raw_orig_offset = df['acc_z_raw'][0]
gyro_x_raw_orig_offset = df['gyro_x_raw'][0]
gyro_y_raw_orig_offset = df['gyro_y_raw'][0]
gyro_z_raw_orig_offset = df['gyro_z_raw'][0]

print(f"Acc Offset: {acc_x_raw_orig_offset}, {acc_y_raw_orig_offset}, {acc_z_raw_orig_offset}")
print(f"Gyro Offset: {gyro_x_raw_orig_offset}, {gyro_y_raw_orig_offset}, {gyro_z_raw_orig_offset}")

acc_x_raw_orig_scale = 2 * (1 + (df['acc_x_raw'][1] + 8) / scale_factor)
acc_y_raw_orig_scale = 2 * (1 + (df['acc_y_raw'][1] + 8) / scale_factor)
acc_z_raw_orig_scale = 2 * (1 + (df['acc_z_raw'][1] + 8) / scale_factor)
gyro_x_raw_orig_scale = (1 + (df['gyro_x_raw'][1] + 8) / scale_factor)
gyro_y_raw_orig_scale = (1 + (df['gyro_y_raw'][1] + 8) / scale_factor)
gyro_z_raw_orig_scale = (1 + (df['gyro_z_raw'][1] + 8) / scale_factor)

print(f"Acc Scale: {acc_x_raw_orig_scale}, {acc_y_raw_orig_scale}, {acc_z_raw_orig_scale}")
print(f"Gyro Scale: {gyro_x_raw_orig_scale}, {gyro_y_raw_orig_scale}, {gyro_z_raw_orig_scale}")

df=df.iloc[2:]

df['acc_x'] = df['acc_x_raw'] / range
df['acc_y'] = df['acc_y_raw'] / range
df['acc_z'] = df['acc_z_raw'] / range
df['acc_x_raw_unoffset'] = df['acc_x_raw'].astype(float) / acc_x_raw_orig_scale - acc_x_raw_orig_offset
df['acc_y_raw_unoffset'] = df['acc_y_raw'].astype(float) / acc_y_raw_orig_scale - acc_y_raw_orig_offset
df['acc_z_raw_unoffset'] = df['acc_z_raw'].astype(float) / acc_z_raw_orig_scale - acc_z_raw_orig_offset
df['acc_x_unoffset'] = df['acc_x_raw_unoffset'] / range
df['acc_y_unoffset'] = df['acc_y_raw_unoffset'] / range
df['acc_z_unoffset'] = df['acc_z_raw_unoffset'] / range

df['gyro_x'] = df['gyro_x_raw'] / range
df['gyro_y'] = df['gyro_y_raw'] / range
df['gyro_z'] = df['gyro_z_raw'] / range
df['gyro_x_raw_unoffset'] = df['gyro_x_raw'] / gyro_x_raw_orig_scale - gyro_x_raw_orig_offset
df['gyro_y_raw_unoffset'] = df['gyro_y_raw'] / gyro_y_raw_orig_scale - gyro_y_raw_orig_offset
df['gyro_z_raw_unoffset'] = df['gyro_z_raw'] / gyro_z_raw_orig_scale - gyro_z_raw_orig_offset
df['gyro_x_unoffset'] = df['gyro_x_raw_unoffset'] / range
df['gyro_y_unoffset'] = df['gyro_y_raw_unoffset'] / range
df['gyro_z_unoffset'] = df['gyro_z_raw_unoffset'] / range

# Accelerometer

In [ ]:
def magnitude(x, y, z):
    return np.sqrt(x**2 + y**2 + z**2)

def plot_acc(x:str, y:str, z:str, title:str, figsize=(12, 6) ):
    fig, axs = plt.subplots(2, 1, figsize=figsize)
    axs[0].plot( df[x], label=x)
    axs[0].plot( df[y], label=y)
    axs[0].plot( df[z], label=z)
    axs[0].set_xlabel('Sample')
    axs[0].set_title(title)
    axs[0].grid(True)
    axs[0].legend()

    axs[1].plot( magnitude(df[x],df[y], df[z]))
    axs[1].set_xlabel('Sample')
    axs[1].set_title('Magnitude')
    axs[1].grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_acc('acc_x','acc_y','acc_z','Accelerometer Data')
magnitude(df['acc_x'], df['acc_y'], df['acc_z']).describe()

In [ ]:
plot_acc('acc_x_unoffset','acc_y_unoffset','acc_z_unoffset','Accelerometer Data')
magnitude(df['acc_x_unoffset'], df['acc_y_unoffset'], df['acc_z_unoffset']).describe()

## Optimize

In [ ]:
def acc_error(theta, x, y, z):
    return ((range)**2 - ((2 * ( 1 + (theta[3] + 8) / scale_factor) *(x + theta[0]))**2 + (2 * ( 1 + (theta[4] + 8) / scale_factor) *(y + theta[1]))**2 + (2 * ( 1 + (theta[5] + 8) / scale_factor) *(z + theta[2]))**2))

# Initial parameter guess
x_0 = np.array([
    acc_x_raw_orig_offset,
    acc_y_raw_orig_offset,
    acc_z_raw_orig_offset,
    acc_x_raw_orig_scale,
    acc_y_raw_orig_scale,
    acc_z_raw_orig_scale,
])

bounds = np.array([
    [-np.inf, -np.inf, -np.inf, -8, -8, -8],
    [np.inf, np.inf, np.inf, 8, 8, 8]
])

# Compute solution providing initial guess theta0, x input, and y input
sol = least_squares(
    fun = acc_error, 
    x0 = x_0, 
    bounds=bounds, 
    args=(df['acc_x_raw_unoffset'],df['acc_y_raw_unoffset'],df['acc_z_raw_unoffset']), 
    method='trf', # trf, dogbox, lm
    loss='linear', # linear, soft_l1, huber, cauchy, arctan
    verbose = 2,
)

acc_x_raw_offset = sol.x[0]
acc_y_raw_offset = sol.x[1]
acc_z_raw_offset = sol.x[2]
acc_x_raw_scale = 2 * (1 + (sol.x[3] + 8) / scale_factor)
acc_y_raw_scale = 2 * (1 + (sol.x[4] + 8) / scale_factor)
acc_z_raw_scale = 2 * (1 + (sol.x[5] + 8) / scale_factor)

df["acc_x_raw_corr"] = (df['acc_x_raw_unoffset'] + acc_x_raw_offset) * acc_x_raw_scale
df["acc_y_raw_corr"] = (df['acc_y_raw_unoffset'] + acc_y_raw_offset) * acc_y_raw_scale
df["acc_z_raw_corr"] = (df['acc_z_raw_unoffset'] + acc_z_raw_offset) * acc_z_raw_scale

df["acc_x_corr"] = df["acc_x_raw_corr"] / range
df["acc_y_corr"] = df["acc_y_raw_corr"] / range
df["acc_z_corr"] = df["acc_z_raw_corr"] / range

plot_acc('acc_x_corr','acc_y_corr','acc_z_corr','Corrected Accelerometer Data')

print(f"NEW OFFSET: {int(acc_x_raw_offset)}, {int(acc_y_raw_offset)}, {int(acc_z_raw_offset)}")
print(f"NEW SCALE: {int(sol.x[3])}, {int(sol.x[4])}, {int(sol.x[5])}")


# Gyroscope

In [ ]:
def plot_gyro(x:str, y:str, z:str, title:str, figsize=(12, 3) ):
    plt.figure(figsize=figsize)
    plt.plot(df[x], label=x)
    plt.plot(df[y], label=y)
    plt.plot(df[z], label=z)
    plt.xlabel('Sample')
    plt.ylabel('Angular Velocity')
    plt.title(title)
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()

In [ ]:
plot_gyro("gyro_x", "gyro_y", "gyro_z", "Gyro Data")

In [ ]:
gyro_x_raw_offset = -df['gyro_x_raw_unoffset'].mean()
gyro_y_raw_offset = -df['gyro_y_raw_unoffset'].mean()
gyro_z_raw_offset = -df['gyro_z_raw_unoffset'].mean()

df['gyro_x_corr'] = (df['gyro_x_raw_unoffset'] + gyro_x_raw_offset) * gyro_x_raw_orig_scale
df['gyro_y_corr'] = (df['gyro_y_raw_unoffset'] + gyro_y_raw_offset) * gyro_y_raw_orig_scale
df['gyro_z_corr'] = (df['gyro_z_raw_unoffset'] + gyro_z_raw_offset) * gyro_z_raw_orig_scale

plot_gyro("gyro_x_corr", "gyro_y_corr", "gyro_z_corr", "Corrected Gyro Data")

print(f"NEW OFFSET: {int(gyro_x_raw_offset)}, {int(gyro_y_raw_offset)}, {int(gyro_z_raw_offset)}")